# Text scraping Labour party leaders speeches

Using beautifulsoup package from http://britishpoliticalspeech.org/speech-archive.htm?q=leader&speaker=&party=4&searchRangeFrom=1895&searchRangeTo=2017

In [13]:
# import modules
from bs4 import BeautifulSoup
import urllib
import re
import pandas as pd

In [14]:
link = 'http://britishpoliticalspeech.org/speech-archive.htm?q=leader&speaker=&party=4&searchRangeFrom=1895&searchRangeTo=2017'

In [15]:
# open url
page = urllib.request.urlopen(link)

# give soup url to parse
soup = BeautifulSoup(page,'html.parser')

test1 = soup.find_all('a')

In [16]:
speech_links = test1[15:-6]

speech_links[:5]

[<a href="speech-archive.htm?speech=366">Leader's speech, Brighton 2017</a>,
 <a href="speech-archive.htm?speech=363">Leader's speech, Liverpool 2016</a>,
 <a href="speech-archive.htm?speech=359">Leader's speech, Brighton 2015</a>,
 <a href="speech-archive.htm?speech=355">Leader's speech, Manchester 2014</a>,
 <a href="speech-archive.htm?speech=353">Leader's speech, Brighton 2013</a>]

In [17]:
dictn = {'Speech' : [],
         'Year' : [],
        'Link' : []}

for i in range(len(speech_links)):
    key = re.search('>(.*)<', str(speech_links[i])).group(1)
    val = re.search('"(.*)"', str(speech_links[i])).group(1)
    link = 'http://britishpoliticalspeech.org/'+val
    dictn['Speech'].append(key)
    dictn['Year'].append(int(key[-4:]))
    dictn['Link'].append(link)

In [18]:
df = pd.DataFrame.from_dict(dictn)

df.head()

,Speech,Year,Link
0,"Leader's speech, Brighton 2017",2017,http://britishpoliticalspeech.org/speech-archi...
1,"Leader's speech, Liverpool 2016",2016,http://britishpoliticalspeech.org/speech-archi...
2,"Leader's speech, Brighton 2015",2015,http://britishpoliticalspeech.org/speech-archi...
3,"Leader's speech, Manchester 2014",2014,http://britishpoliticalspeech.org/speech-archi...
4,"Leader's speech, Brighton 2013",2013,http://britishpoliticalspeech.org/speech-archi...


### Section scraping speeches themselves

In [19]:
link = 'http://britishpoliticalspeech.org/speech-archive.htm?speech=363'

# open url
page = urllib.request.urlopen(link)

# give soup url to parse
soup = BeautifulSoup(page,'html.parser')

test1 = soup.find('div', {'class' : ['speech-content']})

test1 = test1.text

In [20]:
# remove unicode newline char
test1 = test1.replace('\xa0\n', ' ')

test1

'Thank you for that introduction. And how brilliant it is to see the hall here in Liverpool, absolutely packed for the Labour conference, well I say it\'s packed but Virgin Trains assure me there are 800 empty seats.\n Either way Conference, it’s a huge pleasure to be holding our party’s annual gathering here in this fantastic city that has shaped our country, our economy, our culture and our music.\n Liverpool and its people have always been central to the Labour party and our movement. And I know some people say campaigns and protests don’t change things. But the Hillsborough families have shown just how wrong that is.\n It’s taken twenty-seven years but those families have, with great courage and dignity, finally got some truth and justice for the ninety-six who died. And I want to pay tribute to all the families and campaigners, for their solidarity, their commitment and their love.\n We must learn from them so we promise those campaigning for Orgreave, for Shrewsbury, for the thou

In [21]:
dict_text = {'Speech_text' : [],
        'Link' : []}

for i in df['Link']:
    link = i
    dict_text['Link'].append(link)
    # open url
    page = urllib.request.urlopen(link)
    # give soup url to parse
    soup = BeautifulSoup(page,'html.parser')
    # find actual speech content
    speech = soup.find('div', {'class' : ['speech-content']})
    # extract text
    speech = speech.text
    # remove weird unicode newline char
    speech = speech.replace('\xa0\n', ' ')
    dict_text['Speech_text'].append(speech)
    
df_text = pd.DataFrame.from_dict(dict_text)

df_text.head()

,Speech_text,Link
0,"We meet here this week as a united Party, adva...",http://britishpoliticalspeech.org/speech-archi...
1,Thank you for that introduction. And how brill...,http://britishpoliticalspeech.org/speech-archi...
2,"Friends, thank you so much for that incredible...",http://britishpoliticalspeech.org/speech-archi...
3,It’s great to be at Labour Party Conference in...,http://britishpoliticalspeech.org/speech-archi...
4,It's great to be in Brighton. And I want to st...,http://britishpoliticalspeech.org/speech-archi...


In [22]:
combined_df = pd.merge(df,df_text, on='Link')

In [23]:
combined_df

,Speech,Year,Link,Speech_text
0,"Leader's speech, Brighton 2017",2017,http://britishpoliticalspeech.org/speech-archi...,"We meet here this week as a united Party, adva..."
1,"Leader's speech, Liverpool 2016",2016,http://britishpoliticalspeech.org/speech-archi...,Thank you for that introduction. And how brill...
2,"Leader's speech, Brighton 2015",2015,http://britishpoliticalspeech.org/speech-archi...,"Friends, thank you so much for that incredible..."
3,"Leader's speech, Manchester 2014",2014,http://britishpoliticalspeech.org/speech-archi...,It’s great to be at Labour Party Conference in...
4,"Leader's speech, Brighton 2013",2013,http://britishpoliticalspeech.org/speech-archi...,It's great to be in Brighton. And I want to st...
5,"Leader's speech, Manchester 2012",2012,http://britishpoliticalspeech.org/speech-archi...,It is great to be in Labour Manchester. And yo...
6,"Leader's speech, Liverpool 2011",2011,http://britishpoliticalspeech.org/speech-archi...,Thank you Conference.\n It’s great to be in Li...
7,"Leader's speech, Manchester 2010",2010,http://britishpoliticalspeech.org/speech-archi...,"The New Generation\nConference, I stand here t..."
8,"Leader's speech, Brighton 2009",2009,http://britishpoliticalspeech.org/speech-archi...,"You know friends, it is the fighters and belie..."
9,"Leader's speech, Manchester 2008",2008,http://britishpoliticalspeech.org/speech-archi...,"I want to talk with you today about who I am, ..."


In [24]:
combined_df.to_pickle('labour_speeches.pkl')